In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from datetime import datetime
from sklearn.metrics import mean_squared_error

In [2]:
# 1) Leer todos los archivos y mergearlos en un solo dataset
def cargar_y_combinar_datos():
    # Cargar archivos
    sell_in = pd.read_csv('sell-in.txt', sep='\t')
    stocks = pd.read_csv('tb_stocks.txt', sep='\t')
    productos = pd.read_csv('tb_productos.txt', sep='\t')
    #drop duplicates in productos
    productos = productos.drop_duplicates(subset=['product_id'])
    # drop column description from productos
    productos = productos.drop(columns=['descripcion'], errors='ignore')
    
    # Unir datasets
    df = sell_in.merge(stocks, on=['periodo', 'product_id'], how='left')
    df = df.merge(productos, on='product_id', how='left')
    
    return df

def cargar_datos():
    df = pd.read_csv('sell-in.txt', sep='\t')
    return df

def combinar_datos(df):
    stocks = pd.read_csv('tb_stocks.txt', sep='\t')
    productos = pd.read_csv('tb_productos.txt', sep='\t')
    #drop duplicates in productos
    productos = productos.drop_duplicates(subset=['product_id'])
    # drop column description from productos
    productos = productos.drop(columns=['descripcion'], errors='ignore')

    # Unir datasets
    df = df.merge(stocks, on=['periodo', 'product_id'], how='left')
    df = df.merge(productos, on='product_id', how='left')
    
    return df


# 2) Transformar periodo en date
def transformar_periodo(df):
    df['fecha'] = pd.to_datetime(df['periodo'].astype(str), format='%Y%m')
    return df


# 3) Rellenar datos faltantes para series temporales
def completar_series_temporales(df):
    import pandas as pd

    # Asegurar que 'fecha' es Period[M]
    df['fecha'] = df['fecha'].astype('period[M]')
    # ordeno por fecha
    df = df.sort_values(by=['product_id', 'customer_id', 'fecha'])

    columnas_forward_fill = ['plan_precios_cuidados', 'stock_final', 'cat1', 'cat2', 'cat3', 'brand', 'sku_size']
    columnas_a_rellenar = ['cust_request_qty', 'cust_request_tn', 'tn']

    ## 1. Obtener todos los valores únicos de producto, cliente y rango de fechas completo
    all_product_customer = df[['product_id', 'customer_id']].drop_duplicates()
    fecha_min = df['fecha'].min()
    fecha_max = df['fecha'].max()
    todas_fechas = pd.period_range(start=fecha_min, end=fecha_max, freq='M')

    # 2. Crear el cartesian product de (product_id, customer_id, fecha)
    full_index = (
        all_product_customer
        .assign(key=1)
        .merge(pd.DataFrame({'fecha': todas_fechas, 'key': 1}), on='key')
        .drop('key', axis=1)
    )

    # 3. Merge con el dataframe original para obtener datos completos
    df_full = full_index.merge(df, on=['product_id', 'customer_id', 'fecha'], how='left')

    # 4. Ordenar correctamente
    df_full = df_full.sort_values(['product_id', 'customer_id', 'fecha'])

    # 5. Forward fill para columnas deseadas (por grupo)
    df_full[columnas_forward_fill] = (
        df_full
        .groupby(['product_id', 'customer_id'])[columnas_forward_fill]
        .ffill()
    )

    # 6. Rellenar valores faltantes de demanda con 0
    df_full[columnas_a_rellenar] = df_full[columnas_a_rellenar].fillna(0)

    # drop rows where precios_cuidados es nan porque significa que son fechas que no existia ese cliente
    df_full = df_full.dropna(subset=['plan_precios_cuidados'])

    return df_full.reset_index(drop=True)


#4) Crear variable objetivo (t+2)
def crear_variable_objetivo(df):
    # Ordenamos por producto, cliente y fecha
    df = df.sort_values(['product_id', 'customer_id', 'fecha'])
    
    ## Creamos un dataframe para el shift
    #df_shift = df[['product_id', 'customer_id', 'fecha', 'tn']].copy()
    
    # Shift de 2 meses para el target
    #df_shift['fecha_target'] = df_shift['fecha'] + pd.DateOffset(months=2)
    #df_shift.rename(columns={'tn': 'target', 'fecha': 'fecha_origen'}, inplace=True)
    
    ## Unimos con el dataframe original
    #df = df.merge(df_shift[['product_id', 'customer_id', 'fecha_target', 'target']], 
    #             left_on=['product_id', 'customer_id', 'fecha'], 
    #             right_on=['product_id', 'customer_id', 'fecha_target'], 
    #             how='left')
    # el codigo de arriba esta mal, hay que hacer el shift 2 (ya que esta rellenado con 0s), fecha_target no es necesario
    df['target'] = df.groupby(['product_id', 'customer_id'])['tn'].shift(-2)

    return df


# 5) Crear features para el modelo
def crear_features(df, lag_columns=["tn", "cust_request_qty"]):
    # Extraer mes del año
    df['mes'] = df['fecha'].dt.month
    df["year"] = df["fecha"].dt.year
    

    # ratio between cust_request_qty and cust_request_tn (fill Na with 0)
    df["cust_request_ratio"] = df["cust_request_qty"] / df["cust_request_tn"]
    df["cust_request_ratio"] = df["cust_request_ratio"].fillna(0)

    # Crear lags (valores anteriores)
    for i in [1,2,6,12]:  # Lags de 1 a 6 meses
        print(f"Creando lag {i}")
        for col in lag_columns:
            df[f'{col}_lag_{i}'] = df.groupby(['product_id', 'customer_id'])[col].shift(i)
    
    # Crear medias móviles
    for n in [3, 6]:  # Medias de 3, 6 y 12 meses
        print(f"Creando media {n}")
        for col in lag_columns:
            df[f'{col}_media_{n}'] = df.groupby(['product_id', 'customer_id'])[col].transform(
                lambda x: x.rolling(window=n, min_periods=n).mean()
            )
    
    # Crear máximos y mínimos móviles
    for n in [3]:
        for col in lag_columns:
            print(f"Creando max {n}")
            df[f'{col}_max_{n}'] = df.groupby(['product_id', 'customer_id'])[col].transform(
                lambda x: x.rolling(window=n, min_periods=n).max()
            )
            print(f"Creando min {n}")
            df[f'{col}_min_{n}'] = df.groupby(['product_id', 'customer_id'])[col].transform(
                lambda x: x.rolling(window=n, min_periods=n).min()
            )
    
    # Tendencia (diferencia entre media de 3 y 6 meses)
    for col in lag_columns:
        df[f'{col}_tendencia'] = df[f'{col}_media_3'] - df[f'{col}_media_6']
    
    for col in lag_columns:
        df["stock_ratio"] = df["stock_final"] / (df[f"{col}_media_3"] + 1)  # +1 para evitar división por cero
    
    return df 


#6) Separar el dataset
def separar_dataset(df):
    fechas_ordenadas = sorted(df['fecha'].unique())
    ultima_fecha = fechas_ordenadas[-1]
    penultima_fecha = fechas_ordenadas[-3]
    antepenultima_fecha = fechas_ordenadas[-4]
    
    # Dataset para predicción final (Kaggle)
    kaggle_pred = df[df['fecha'] == ultima_fecha].copy()
    
    # Dataset de test
    test = df[df['fecha'] == penultima_fecha].copy()
    
    # Dataset de evaluación
    eval_data = df[df['fecha'] == antepenultima_fecha].copy()
    
    # Dataset de entrenamiento
    train = df[(df['fecha'] < antepenultima_fecha) & (df['fecha'] != ultima_fecha)].copy()
    
    return train, eval_data, test, kaggle_pred



# Creamos una clase para la métrica personalizada que necesita acceso a product_id
class CustomMetric:
    def __init__(self, df_eval, product_id_col='product_id'):
        self.df_eval = df_eval
        self.product_id_col = product_id_col
    
    def __call__(self, preds, train_data):
        labels = train_data.get_label()
        df_temp = self.df_eval.copy()
        df_temp['preds'] = preds
        df_temp['labels'] = labels
        
        # Agrupar por product_id y calcular el error
        por_producto = df_temp.groupby(self.product_id_col).agg({'labels': 'sum', 'preds': 'sum'})
        
        # Calcular el error personalizado
        error = np.sum(np.abs(por_producto['labels'] - por_producto['preds'])) / np.sum(por_producto['labels'])
        
        # LightGBM espera que el segundo valor sea mayor cuando el modelo es mejor
        return 'custom_error', error, False

def entrenar_modelo(X_train, y_train, X_eval, y_eval, df_eval):
    cat_features = [col for col in X_train.columns if X_train[col].dtype.name == 'category']
    train_data = lgb.Dataset(X_train, label=y_train, categorical_feature=cat_features)
    eval_data = lgb.Dataset(X_eval, label=y_eval, reference=train_data, categorical_feature=cat_features)
    
    df_eval_metric = df_eval[['product_id']].copy()
    custom_metric = CustomMetric(df_eval_metric)
    
    params = {
        'objective': 'regression',
        'boosting_type': 'gbdt',
        'learning_rate': 0.05,
        'num_leaves': 31,
        'min_data_in_leaf': 20,
        'feature_fraction': 0.9,
        'bagging_fraction': 0.8,
        'bagging_freq': 5,
        'verbose': -1
    }
    
    callbacks = [
        lgb.early_stopping(50),
        lgb.log_evaluation(100)
    ]
    
    model = lgb.train(
        params,
        train_data,
        num_boost_round=1000,
        valid_sets=[eval_data],
        feval=custom_metric,
        callbacks=callbacks
    )
    return model
    
# 8) Evaluar el modelo en test
def evaluar_modelo(model, X_test, test_df):
    # Predicciones
    predictions = model.predict(X_test)
    test_df['predictions'] = predictions
    
    # Error por producto
    product_actual = test_df.groupby('product_id')['target'].sum()
    product_pred = test_df.groupby('product_id')['predictions'].sum()
    
    # Crear DataFrame de evaluación
    eval_df = pd.DataFrame({
        'product_id': product_actual.index,
        'tn_real': product_actual.values,
        'tn_predicha': product_pred.values
    })
    
    # Calcular el error personalizado
    total_error = np.sum(np.abs(eval_df['tn_real'] - eval_df['tn_predicha'])) / np.sum(eval_df['tn_real'])
    
    print(f"Error en test: {total_error:.4f}")
    print("\nTop 5 productos con mayor error absoluto:")
    eval_df['error_absoluto'] = np.abs(eval_df['tn_real'] - eval_df['tn_predicha'])
    print(eval_df.sort_values('error_absoluto', ascending=False).head())
    
    return eval_df, total_error


# Función para optimizar el dataframe y reducir memoria
def optimizar_dataframe(df):
    # Copia para no modificar el original
    df_optimizado = df.copy()
    
    # Convertir float64 a float32
    for col in df_optimizado.select_dtypes(include=['float64']).columns:
        df_optimizado[col] = df_optimizado[col].astype('float32')
    
    # Optimizar fecha - usar periodo_date con solo mes y año
    if 'fecha' in df_optimizado.columns:
        # Usar un formato más ligero para fecha (solo mes y año)
        df_optimizado['fecha'] = pd.PeriodIndex(df_optimizado['fecha'], freq='M')
    
    # Nota: Mantener 'periodo' ya que se usa para operaciones posteriores
    # pero se podría eliminar al final del proceso si ya no es necesario
    
    # Convertir customer_id y product_id a int32
    if 'customer_id' in df_optimizado.columns:
        df_optimizado['customer_id'] = df_optimizado['customer_id'].astype("uint32")
    
    if 'product_id' in df_optimizado.columns:
        df_optimizado['product_id'] = df_optimizado['product_id'].astype("uint32")
    
    # Convertir plan_precios_cuidados a categoría
    if 'plan_precios_cuidados' in df_optimizado.columns:
        df_optimizado['plan_precios_cuidados'] = df_optimizado['plan_precios_cuidados'].astype('category')
    
    # Convertir cust_request_qty a int32 si es posible
    if 'cust_request_qty' in df_optimizado.columns:
        if df_optimizado['cust_request_qty'].dropna().apply(lambda x: float(x).is_integer()).all():
            df_optimizado['cust_request_qty'] = df_optimizado['cust_request_qty'].fillna(0).astype('int32')
        else:
            df_optimizado['cust_request_qty'] = df_optimizado['cust_request_qty'].astype('float32')
    
    # Convertir columnas de objeto a categorías
    for col in df_optimizado.select_dtypes(include=['object']).columns:
        df_optimizado[col] = df_optimizado[col].astype('category')
    
    # Mostrar información sobre la reducción de memoria
    print(f"Memoria antes de optimizar: {df.memory_usage().sum() / 1024**2:.2f} MB")
    print(f"Memoria después de optimizar: {df_optimizado.memory_usage().sum() / 1024**2:.2f} MB")
    
    return df_optimizado


def completar_series_temporales_v1(df):
    """
    Esta version completa las series temporales de productos y clientes cuando AMBOS estaban activos. 
    Entiendo ACTIVOS como los periodos que son igual o mayores a la primer fecha de venta de cada cliente y producto.
    """
    import pandas as pd

    # Asegurar que 'fecha' es Period[M]
    df['fecha'] = df['fecha'].astype('period[M]')

    columnas_forward_fill = ['plan_precios_cuidados', 'stock_final', 'cat1', 'cat2', 'cat3', 'brand', 'sku_size']
    columnas_a_rellenar = ['cust_request_qty', 'cust_request_tn', 'tn']

    # 1. Fecha mínima de aparición por cliente y producto
    fecha_ini_clientes = df.groupby('customer_id')['fecha'].min().reset_index().rename(columns={'fecha': 'fecha_ini_c'})
    fecha_ini_productos = df.groupby('product_id')['fecha'].min().reset_index().rename(columns={'fecha': 'fecha_ini_p'})

    # 2. Rango completo de fechas
    fechas = pd.period_range(df['fecha'].min(), df['fecha'].max(), freq='M')
    fechas_df = pd.DataFrame({'fecha': fechas})

    # 3. Combinar clientes x fechas >= fecha_ini
    clientes_fechas = fecha_ini_clientes.merge(fechas_df, how='cross')
    clientes_fechas = clientes_fechas[clientes_fechas['fecha'] >= clientes_fechas['fecha_ini_c']]
    clientes_fechas = clientes_fechas[['customer_id', 'fecha']]

    productos_fechas = fecha_ini_productos.merge(fechas_df, how='cross')
    productos_fechas = productos_fechas[productos_fechas['fecha'] >= productos_fechas['fecha_ini_p']]
    productos_fechas = productos_fechas[['product_id', 'fecha']]

    # 4. Intersección cliente-producto donde ambos ya eran activos (pero sin cortar por fecha final)
    posibles_combinaciones = productos_fechas.merge(clientes_fechas, on='fecha', how='inner')

    # 5. Merge con datos originales
    df_full = posibles_combinaciones.merge(df, on=['product_id', 'customer_id', 'fecha'], how='left')

    # 6. Ordenar
    df_full = df_full.sort_values(['product_id', 'customer_id', 'fecha'])

    # 7. Forward fill por grupo
    df_full[columnas_forward_fill] = (
        df_full.groupby(['product_id', 'customer_id'])[columnas_forward_fill].ffill()
    )

    # 8. Completar demanda con 0
    df_full[columnas_a_rellenar] = df_full[columnas_a_rellenar].fillna(0)

    # dropea las rows donde el el producto no existia
    stocks = pd.read_csv('tb_stocks.txt', sep='\t')
    productos = pd.read_csv('tb_productos.txt', sep='\t')
    #drop duplicates in productos
    productos = productos.drop_duplicates(subset=['product_id'])
    productos = productos.drop(columns=['descripcion'], errors='ignore')

    # Unir datasets
    # drop stock_final y cat1	cat2	cat3	brand	sku_size	ya que se hacen en el merge
    df_full = df_full.drop(columns=['stock_final', 'cat1', 'cat2', 'cat3', 'brand', 'sku_size'])
    df_full = df_full.merge(stocks, on=['periodo', 'product_id'], how='left')
    df_full = df_full.merge(productos, on='product_id', how='left')
    
    return df_full.reset_index(drop=True)

def completar_series_temporales_v2(df):
    """
    Esta versión completa las series temporales de productos y clientes cuando AMBOS estaban activos.
    Se considera 'ACTIVO' el periodo entre la primera y última fecha de venta de cada cliente y producto.
    """
    import pandas as pd

    # Asegurar que 'fecha' es Period[M]
    df['fecha'] = pd.to_datetime(df['periodo'].astype(str), format='%Y%m')
    df['fecha'] = df['fecha'].astype('period[M]')

    columnas_forward_fill = ['plan_precios_cuidados']
    columnas_a_rellenar = ['cust_request_qty', 'cust_request_tn', 'tn', 'plan_precios_cuidados']

    # 1. Fecha mínima y máxima de aparición por cliente y producto
    fechas_clientes = df.groupby('customer_id')['fecha'].agg(fecha_ini_c='min', fecha_fin_c='max').reset_index()
    fechas_productos = df.groupby('product_id')['fecha'].agg(fecha_ini_p='min', fecha_fin_p='max').reset_index()

    # los clientes donde la fecha de fin sea menor a 2019-09 lo paso a 2020-01
    fechas_clientes.loc[fechas_clientes['fecha_fin_c'] >= '2019-09', 'fecha_fin_c'] = '2020-01'


    # print number of fechas that are 2020-01
    print(f"Número de clientes con fecha_fin_c en 2020-01: {fechas_clientes[fechas_clientes['fecha_fin_c'] == '2020-01'].shape[0]}")

    # 2. Rango completo de fechas
    fechas = pd.period_range(df['fecha'].min(), df['fecha'].max(), freq='M')
    fechas_df = pd.DataFrame({'fecha': fechas})

    # 3. Crear combinación cliente-fechas activas
    clientes_fechas = fechas_clientes.merge(fechas_df, how='cross')
    clientes_fechas = clientes_fechas[

        (clientes_fechas['fecha'] >= clientes_fechas['fecha_ini_c']) &
        (clientes_fechas['fecha'] <= clientes_fechas['fecha_fin_c'])
    ][['customer_id', 'fecha']]

    # 4. Crear combinación producto-fechas activas
    productos_fechas = fechas_productos.merge(fechas_df, how='cross')
    productos_fechas = productos_fechas[
        (productos_fechas['fecha'] >= productos_fechas['fecha_ini_p']) &
        (productos_fechas['fecha'] <= productos_fechas['fecha_fin_p'])
    ][['product_id', 'fecha']]

    # 5. Intersección cliente-producto donde ambos eran activos
    posibles_combinaciones = productos_fechas.merge(clientes_fechas, on='fecha', how='inner')

    # 6. Merge con datos originales
    df_full = posibles_combinaciones.merge(df, on=['product_id', 'customer_id', 'fecha'], how='left')

    # 7. Ordenar
    df_full = df_full.sort_values(['product_id', 'customer_id', 'fecha'])

    # 8. Forward fill por grupo
    #df_full[columnas_forward_fill] = (
    #    df_full.groupby(['product_id', 'customer_id'])[columnas_forward_fill].ffill()
    #)

    # 9. Completar demanda con 0
    df_full[columnas_a_rellenar] = df_full[columnas_a_rellenar].fillna(0)

    return df_full.reset_index(drop=True)

In [3]:
df = cargar_datos()
df.head()

,periodo,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn
0,201701,10234,20524,0,2,0.05300,0.05300
1,201701,10032,20524,0,1,0.13628,0.13628
2,201701,10217,20524,0,1,0.03028,0.03028
3,201701,10125,20524,0,1,0.02271,0.02271
4,201701,10012,20524,0,11,1.54452,1.54452


In [4]:
df = transformar_periodo(df)
df.head()

,periodo,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,fecha
0,201701,10234,20524,0,2,0.05300,0.05300,2017-01-01
1,201701,10032,20524,0,1,0.13628,0.13628,2017-01-01
2,201701,10217,20524,0,1,0.03028,0.03028,2017-01-01
3,201701,10125,20524,0,1,0.02271,0.02271,2017-01-01
4,201701,10012,20524,0,11,1.54452,1.54452,2017-01-01


In [5]:
df = completar_series_temporales_v2(df)
df.info()

Número de clientes con fecha_fin_c en 2020-01: 500
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15643071 entries, 0 to 15643070
Data columns (total 8 columns):
 #   Column                 Dtype    
---  ------                 -----    
 0   product_id             int64    
 1   fecha                  period[M]
 2   customer_id            int64    
 3   periodo                float64  
 4   plan_precios_cuidados  float64  
 5   cust_request_qty       float64  
 6   cust_request_tn        float64  
 7   tn                     float64  
dtypes: float64(5), int64(2), period[M](1)
memory usage: 954.8 MB


In [6]:
df

,product_id,fecha,customer_id,periodo,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn
0,20001,2017-01,10001,201701.0,0.0,11.0,99.43861,99.43861
1,20001,2017-02,10001,201702.0,0.0,23.0,198.84365,198.84365
2,20001,2017-03,10001,201703.0,0.0,33.0,92.46537,92.46537
3,20001,2017-04,10001,201704.0,0.0,8.0,13.29728,13.29728
4,20001,2017-05,10001,201705.0,0.0,15.0,101.20711,101.00563
...,...,...,...,...,...,...,...,...
15643066,21299,2017-08,10605,NaN,0.0,0.0,0.00000,0.00000
15643067,21299,2017-08,10611,NaN,0.0,0.0,0.00000,0.00000
15643068,21299,2017-08,10614,NaN,0.0,0.0,0.00000,0.00000
15643069,21299,2017-08,10615,NaN,0.0,0.0,0.00000,0.00000


In [7]:
df = combinar_datos(df)

In [8]:
df = optimizar_dataframe(df)
df.info()

Memoria antes de optimizar: 1670.86 MB
Memoria después de optimizar: 671.33 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15643071 entries, 0 to 15643070
Data columns (total 14 columns):
 #   Column                 Dtype    
---  ------                 -----    
 0   product_id             uint32   
 1   fecha                  period[M]
 2   customer_id            uint32   
 3   periodo                float32  
 4   plan_precios_cuidados  category 
 5   cust_request_qty       int32    
 6   cust_request_tn        float32  
 7   tn                     float32  
 8   stock_final            float32  
 9   cat1                   category 
 10  cat2                   category 
 11  cat3                   category 
 12  brand                  category 
 13  sku_size               float32  
dtypes: category(5), float32(5), int32(1), period[M](1), uint32(2)
memory usage: 671.3 MB


In [9]:
# guardo el df intermedio como parquet para que conserve el tipo de dato
# paso plan_precios_cuidados a booleano donde 1 es True y 0 es False
#df['plan_precios_cuidados'] = df['plan_precios_cuidados'].astype(np.uint8)
#df['plan_precios_cuidados'] = df['plan_precios_cuidados'] > 0
df.to_parquet('df_intermedio.parquet')


In [11]:
product_id_a_predecir = pd.read_csv('product_id_apredecir201912.txt', sep='\t')['product_id'].unique()
product_id_a_predecir

array([20001, 20002, 20003, 20004, 20005, 20006, 20007, 20008, 20009,
       20010, 20011, 20012, 20013, 20014, 20015, 20016, 20017, 20018,
       20019, 20020, 20021, 20022, 20023, 20024, 20025, 20026, 20027,
       20028, 20029, 20030, 20031, 20032, 20033, 20035, 20037, 20038,
       20039, 20041, 20042, 20043, 20044, 20045, 20046, 20047, 20049,
       20050, 20051, 20052, 20053, 20054, 20055, 20056, 20057, 20058,
       20059, 20061, 20062, 20063, 20065, 20066, 20067, 20068, 20069,
       20070, 20071, 20072, 20073, 20074, 20075, 20076, 20077, 20079,
       20080, 20081, 20082, 20084, 20085, 20086, 20087, 20089, 20090,
       20091, 20092, 20093, 20094, 20095, 20096, 20097, 20099, 20100,
       20101, 20102, 20103, 20106, 20107, 20108, 20109, 20111, 20112,
       20114, 20116, 20117, 20118, 20119, 20120, 20121, 20122, 20123,
       20124, 20125, 20126, 20127, 20129, 20130, 20132, 20133, 20134,
       20135, 20137, 20138, 20139, 20140, 20142, 20143, 20144, 20145,
       20146, 20148,

In [12]:
# filtrar df por product_id_a_predecir
df = df[df['product_id'].isin(product_id_a_predecir)]
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12254054 entries, 0 to 19250837
Data columns (total 15 columns):
 #   Column                 Dtype    
---  ------                 -----    
 0   product_id             int64    
 1   fecha                  period[M]
 2   customer_id            category 
 3   periodo                float64  
 4   plan_precios_cuidados  category 
 5   cust_request_qty       float64  
 6   cust_request_tn        float32  
 7   tn                     float32  
 8   stock_final            float64  
 9   cat1                   object   
 10  cat2                   object   
 11  cat3                   object   
 12  brand                  object   
 13  sku_size               float64  
 14  target                 float32  
dtypes: category(2), float32(3), float64(4), int64(1), object(4), period[M](1)
memory usage: 1.2+ GB


In [13]:
df

,product_id,fecha,customer_id,periodo,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,stock_final,cat1,cat2,cat3,brand,sku_size,target
0,20001,2017-01,10001,201701.0,0,11.0,99.438606,99.438606,NaN,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,92.465370
1,20001,2017-02,10001,201702.0,0,23.0,198.843643,198.843643,NaN,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,13.297280
2,20001,2017-03,10001,201703.0,0,33.0,92.465370,92.465370,NaN,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,101.005630
3,20001,2017-04,10001,201704.0,0,8.0,13.297280,13.297280,NaN,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,128.047913
4,20001,2017-05,10001,201705.0,0,15.0,101.207108,101.005630,NaN,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,101.207108
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19250833,21276,2019-08,10637,NaN,NaN,0.0,0.000000,0.000000,NaN,PC,PIEL1,Cara,NIVEA,140.0,0.000000
19250834,21276,2019-09,10637,NaN,NaN,0.0,0.000000,0.000000,NaN,PC,PIEL1,Cara,NIVEA,140.0,0.000000
19250835,21276,2019-10,10637,NaN,NaN,0.0,0.000000,0.000000,NaN,PC,PIEL1,Cara,NIVEA,140.0,0.000000
19250836,21276,2019-11,10637,NaN,NaN,0.0,0.000000,0.000000,NaN,PC,PIEL1,Cara,NIVEA,140.0,NaN


In [ ]:
# transformo plan_precios_cuidados a bool (True si > 0  )
df = pd.read_parquet('df_intermedio.parquet')

df.head()


In [ ]:
df.info()

In [ ]:
# transformar product_id y customer_id a category
df['product_id'] = df['product_id'].astype('category')
df['customer_id'] = df['customer_id'].astype('category')
#borro periodo
df = df.drop(columns=['periodo'])
# paso cust_request_qty a int32
df['cust_request_qty'] = df['cust_request_qty'].astype('int32')
df.info()


In [ ]:
df = crear_features(df)
df.head()

In [ ]:
# guardar archivo como parquet para que conserve el tipo de dato
df.to_parquet('df.parquet')


In [ ]:

df = pd.read_parquet('df.parquet')
df.head()

In [ ]:
#sub_df es product_id 20001 customer_id 10010
sub_df = df[(df['product_id'] == 20001) & (df['customer_id'] == 10010)]
print(sub_df.to_string())

In [ ]:
df.info()

In [ ]:
# transformar product_id y customer_id a category
df['product_id'] = df['product_id'].astype('category')
df['customer_id'] = df['customer_id'].astype('category')
# paso mes y year a uint16
df['mes'] = df['mes'].astype('uint16')
df['year'] = df['year'].astype('uint16')

df.info()


In [ ]:
# reemplazar todos los float64 por float32
for col in df.select_dtypes(include=['float64']).columns:
    df[col] = df[col].astype('float32')
df.info()

In [ ]:
# obtengo producto 20001 customer_id 10001 tn y target
df[df['product_id'] == 20001][df['customer_id'] == 10006][['tn', 'target']].plot()


In [ ]:
# 6) Separar dataset
train, eval_data, test, kaggle_pred = separar_dataset(df)

In [ ]:

# Seleccionar columnas para el modelo (excluyendo variables no numéricas y target)

# Seleccionar columnas para el modelo (incluyendo variables categóricas e identificadores)
columnas_modelo = [col for col in train.columns if col not in 
                   ['periodo', 'periodo_original', 'fecha', 'target', 
                    'fecha_origen', 'primera_compra']]

# 7) Entrenar modelo con train+eval
X_train = pd.concat([train[columnas_modelo], eval_data[columnas_modelo]])
y_train = pd.concat([train['target'], eval_data['target']])

X_eval = eval_data[columnas_modelo]
y_eval = eval_data['target']

X_test = test[columnas_modelo]
y_test = test['target']

In [ ]:
y_test

In [ ]:
# Entrenar primer modelo usando nuestra métrica personalizada
# silence FutureWarnings of pandas
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)



modelo = entrenar_modelo(X_train, y_train, X_eval, y_eval, eval_data)

In [ ]:
# 8) Evaluar en test
eval_df,error_test = evaluar_modelo(modelo, X_test, y_test, test)


In [ ]:
print(eval_df.to_string())

In [ ]:
# mostrar features importantes
lgb.plot_importance(modelo)


In [ ]:


# 9) Reentrenar con más datos
X_train_final = pd.concat([train[columnas_modelo], eval_data[columnas_modelo], test[columnas_modelo]])
y_train_final = pd.concat([train['target'], eval_data['target'], test['target']])

# Para el entrenamiento final, usamos test como conjunto de evaluación
modelo_final = entrenar_modelo(X_train_final, y_train_final, X_test, y_test, test)

# 10) Predecir para Kaggle y preparar submission
X_kaggle = kaggle_pred[columnas_modelo]
kaggle_pred['tn_predicha'] = modelo_final.predict(X_kaggle)

# Agrupar por product_id para la submission
submission = kaggle_pred.groupby('product_id')['tn_predicha'].sum().reset_index()
submission.columns = ['product_id', 'tn']

# Guardar archivo de submission
submission.to_csv(f'submission_{error_test}.csv', index=False)
print("Archivo de submission generado correctamente.")

In [ ]:
columnas_modelo

In [ ]:
def entrenar_modelo_dos_etapas(X_train, y_train, X_eval, y_eval, df_eval):
    # Primera etapa: Clasificador para predecir si será cero o no
    y_train_binary = (y_train > 0).astype(int)
    y_eval_binary = (y_eval > 0).astype(int)
    
    # Entrenar clasificador
    params_clf = {
        'objective': 'binary',
        'boosting_type': 'gbdt',
        'learning_rate': 0.05,
        'num_leaves': 31,
        'min_data_in_leaf': 20,
        'feature_fraction': 0.9,
        'bagging_fraction': 0.8,
        'bagging_freq': 5,
        'verbose': -1
    }
    
    train_data_clf = lgb.Dataset(X_train, label=y_train_binary)
    eval_data_clf = lgb.Dataset(X_eval, label=y_eval_binary, reference=train_data_clf)
    
    
    model_clf = lgb.train(
        params_clf,
        train_data_clf,
        num_boost_round=1000,
        valid_sets=[eval_data_clf],
        callbacks=[lgb.early_stopping(50), lgb.log_evaluation(100)]
    )
    
    # Segunda etapa: Regresor para todos los casos
    params_reg = {
        'objective': 'regression',
        'boosting_type': 'gbdt',
        'learning_rate': 0.05,
        'num_leaves': 31,
        'min_data_in_leaf': 20,
        'feature_fraction': 0.9,
        'bagging_fraction': 0.8,
        'bagging_freq': 5,
        'verbose': -1
    }
    
    train_data_reg = lgb.Dataset(X_train, label=y_train)
    eval_data_reg = lgb.Dataset(X_eval, label=y_eval, reference=train_data_reg)
    
    # Métrica custom para regresión
    custom_metric_reg = CustomMetric(df_eval[['product_id']].copy())
    
    model_reg = lgb.train(
        params_reg,
        train_data_reg,
        num_boost_round=1000,
        valid_sets=[eval_data_reg],
        feval=custom_metric_reg,
        callbacks=[lgb.early_stopping(50), lgb.log_evaluation(100)]
    )
    
    return model_clf, model_reg

def evaluar_modelo_dos_etapas(model_clf, model_reg, X_test, y_test, test_df):
    # Primera etapa: Predecir probabilidad de ser no cero
    prob_non_zero = model_clf.predict(X_test)
    
    # Segunda etapa: Predecir valor para todos los casos
    predictions_reg = model_reg.predict(X_test)
    
    # Combinar predicciones: si la probabilidad de ser no cero es baja, forzar a cero
    predictions = np.where(prob_non_zero > 0.5, predictions_reg, 0)
    test_df['predictions'] = predictions
    
    # Error por producto
    product_actual = test_df.groupby('product_id')['target'].sum()
    product_pred = test_df.groupby('product_id')['predictions'].sum()
    
    # Crear DataFrame de evaluación
    eval_df = pd.DataFrame({
        'product_id': product_actual.index,
        'tn_real': product_actual.values,
        'tn_predicha': product_pred.values
    })
    
    # Calcular el error personalizado
    total_error = np.sum(np.abs(eval_df['tn_real'] - eval_df['tn_predicha'])) / np.sum(eval_df['tn_real'])
    
    print(f"Error en test: {total_error:.4f}")
    print("\nTop 5 productos con mayor error absoluto:")
    eval_df['error_absoluto'] = np.abs(eval_df['tn_real'] - eval_df['tn_predicha'])
    print(eval_df.sort_values('error_absoluto', ascending=False).head())
    
    # Métricas adicionales
    print("\nMétricas adicionales:")
    print(f"Porcentaje de ceros reales: {(eval_df['tn_real'] == 0).mean()*100:.2f}%")
    print(f"Porcentaje de ceros predichos: {(eval_df['tn_predicha'] == 0).mean()*100:.2f}%")
    
    # Error solo en casos no cero
    mask_non_zero = eval_df['tn_real'] > 0
    error_non_zero = np.sum(np.abs(eval_df.loc[mask_non_zero, 'tn_real'] - 
                                  eval_df.loc[mask_non_zero, 'tn_predicha'])) / \
                     np.sum(eval_df.loc[mask_non_zero, 'tn_real'])
    print(f"Error en casos no cero: {error_non_zero:.4f}")
    
    return eval_df, total_error

In [ ]:
# Entrenar modelos
model_clf, model_reg = entrenar_modelo_dos_etapas(X_train, y_train, X_eval, y_eval, eval_data)


In [ ]:
eval_df, total_error = evaluar_modelo_dos_etapas(model_clf, model_reg, X_test, y_test, test)

In [ ]:
print(eval_df.to_string())

In [ ]:
lgb.plot_importance(model_clf)
lgb.plot_importance(model_reg)

In [ ]:
# 9) Reentrenar con más datos
X_train_final = pd.concat([train[columnas_modelo], eval_data[columnas_modelo], test[columnas_modelo]])
y_train_final = pd.concat([train['target'], eval_data['target'], test['target']])

# Para el entrenamiento final, usamos test como conjunto de evaluación
modelo_clf_final, modelo_reg_final = entrenar_modelo_dos_etapas(X_train_final, y_train_final, X_test, y_test, test)

# 10) Predecir para Kaggle y preparar submission
X_kaggle = kaggle_pred[columnas_modelo]

# Primera etapa: Predecir probabilidad de ser no cero
prob_non_zero = modelo_clf_final.predict(X_kaggle)

# Segunda etapa: Predecir valor
predictions_reg = modelo_reg_final.predict(X_kaggle)

# Combinar predicciones: si la probabilidad de ser no cero es baja, forzar a cero
kaggle_pred['tn_predicha'] = np.where(prob_non_zero > 0.5, predictions_reg, 0)

# Agrupar por product_id para la submission
submission = kaggle_pred.groupby('product_id')['tn_predicha'].sum().reset_index()
submission.columns = ['product_id', 'tn']

# Guardar archivo de submission
submission.to_csv(f'submission_{total_error}_2_etapas.csv', index=False)
print("Archivo de submission generado correctamente.")